In [36]:
import os
import pathlib
import tensorflow as tf

from PIL import Image
from metrics import dice_coef, dice_loss, jaccard_distance

### best models RGB

In [70]:
image_size = 400
color_mode = 1
# unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/RGB/19-10-2022-12-25-10/4/unet.h5' # 256 RGB
# unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/RGB/19-10-2022-12-55-58/3/unet.h5' # 400 RGB
# unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/RGB/19-10-2022-13-20-10/4/unet.h5' # 512 RGB

### best models GRAYSCALE

In [71]:
# unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/grayscale/19-10-2022-09-34-27/3/unet.h5' # 256 GRAYSCALE
unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/grayscale/19-10-2022-09-55-49/3/unet.h5' # 400 GRAYSCALE
# unet_model = '/home/xandao/Documentos/resultados_gimp/u-net/grayscale/19-10-2022-10-53-51/3/unet.h5' # 512 GRAYSCALE

In [72]:
path_imgs = f'/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/originais/{image_size}/jpeg'
out_folder = f'/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/{image_size}'
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

list_images = sorted([file for file in pathlib.Path(path_imgs).rglob('*')])

os.path.isfile(unet_model), os.path.exists(path_imgs), len(list_images)

(True, True, 1419)

In [73]:
for d in ['mask_unet', 'w_pred_mask', 'transparency']:
    p = os.path.join(out_folder, d)
    print(p)
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

model = tf.keras.models.load_model(unet_model, custom_objects = {'dice_loss': dice_loss, 'dice_coef': dice_coef, 'jaccard_distance': jaccard_distance })

for i, file in enumerate(list_images):
    print(i, file.resolve())
    if color_mode == 1:
        image_original = tf.keras.preprocessing.image.load_img(file.resolve(), color_mode='grayscale')
    else:
        image_original = tf.keras.preprocessing.image.load_img(file.resolve())
    image = tf.keras.preprocessing.image.img_to_array(image_original)
    image = image / 255
    image = image.reshape((1, image_size, image_size, color_mode))
    mask = model.predict(image)
    mask = mask[0, :, :, :]
    new_filename = os.path.join(out_folder, 'mask_unet', str(file.stem) + '.bmp')
    print(new_filename, mask.shape)
    tf.keras.preprocessing.image.save_img(new_filename, mask)

    mask = tf.keras.preprocessing.image.array_to_img(mask).convert('L')
    image_original = image_original.convert('RGBA')
    new_filename = os.path.join(out_folder, 'transparency', str(file.stem) + '_transparente.png')
    image_original.putalpha(mask)
    image_original.save(new_filename)

    background = Image.new('RGBA', (image_size, image_size), 'WHITE')
    img_w, img_h = image_original.size
    bg_w, bg_h = background.size
    offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
    background.paste(image_original, offset, image_original)
    new_filename = os.path.join(out_folder, 'w_pred_mask', str(file.stem) + '.jpeg')
    background = background.convert('RGB')
    background.save(new_filename)

/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/400/mask_unet
/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/400/w_pred_mask
/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/400/transparency
0 /home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/originais/400/jpeg/ALCB002267.jpeg
1/1 [==============================] - 0s 211ms/step
/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/400/mask_unet/ALCB002267.bmp (400, 400, 1)
1 /home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/originais/400/jpeg/ALCB002274.jpeg
1/1 [==============================] - 0s 14ms/step
/home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grayscale/segmented_unet/400/mask_unet/ALCB002274.bmp (400, 400, 1)
2 /home/xandao/Documentos/GitHub/dataset_gimp/imagens_george/imagens/grays